# ETL Siniestros viales Argentina
En este notebook, nos encargaremos de dejar el set de datos de los homicidios que ocurrieron en los siniestros viales limpio, para su posterior análisis y uso en KPIs.

# Importamos las librerías necesarias

In [1]:
import pandas as pd # Trataremos los datos con pandas.
from difflib import SequenceMatcher # Compararemos cadenas de texto con SequenceMatcher.
from IPython.display import clear_output # Limpiamos las salidas de texto.
import os # Verificaremos archivos con OS.
import json # Archivos JSON en python.

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_26472\3655460051.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # Trataremos los datos con pandas.


# 1. Carga de datos y previsualización

El set de datos se divide en 2 hojas, siendo estas hechos y victimas, trataremos estas dos hojas y las visualizaremos juntas para hallar la relación que tienen rápidamente y saber por donde empezar.

Carguemos los datos.

In [2]:
# Datos de Hechos.
df_h = pd.read_excel('../Data/homicidios.xlsx', sheet_name=0)
# Datos de Victimas.
df_v = pd.read_excel('../Data/homicidios.xlsx', sheet_name=2)

## 1.1 Tratamiento valores faltantes adicionales
Consultando antes el diccionario de datos, se detectó que los valores faltantes en el set de datos, también se representan como SD, y como un punto para las columnas de coordenadas, por lo que reemplazaremos eso por NANs para la correcta visualización de nulos.

In [3]:
no_data = ['SD', 'Sd', 'sD', '.'] # Definimos los posibles SD (Sin Dato) y puntos que puede haber dentro del df.
df_h = df_h.replace(no_data, pd.NA) # Los reemplazamos por un valor nulo que pandas entienda como nulo.
df_v = df_v.replace(no_data, pd.NA)

## 1.2 Previsualización

Empezamos viendo los valores faltantes de cada DataFrame.

In [4]:
df_h.isna().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       1
HH                         1
LUGAR_DEL_HECHO            1
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                     12
pos y                     12
PARTICIPANTES              0
VICTIMA                    9
ACUSADO                   23
dtype: int64

In [5]:
df_v.isna().sum()

ID_hecho                0
FECHA                   0
AAAA                    0
MM                      0
DD                      0
ROL                    11
VICTIMA                 9
SEXO                    6
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64

Ahora observemos los duplicados. Unicamente del set de homicidios, ya que el set de victimas debe tener duplicados al ser más de 1 victima la implicada.

In [6]:
df_h[df_h.duplicated(keep=False)]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO


No hay valores duplicados en el set de datos.

# 2 Limpieza nulos y columnas irrelevantes

Viendo el set de datos antes, y con esta primera vista, podemos observar que hay varias columnas redundantes o que no aportarán nada a el análisis, así que empezaremos eliminando dichas columnas y explicaremos el criterio de eliminación.

**FECHA y DD(día):** Únicamente utilizaré el **mes** y el **año** para graficar la distribución de accidentes, para esto ya cuento con la columna **AAAA** y **MM**, por lo que tanto la columna **FECHA** y **DD(día)** no me serán relevantes para esta tarea, en **ambos** datasets.  
**HORA:** La hora en su totalidad, con sus minutos, no la usaré para describir alguna distribución, mientras que la hora en su número entero, sí, esta hora entera se encuentra en la columna **HH**.  
**ALTURA y CRUCE:** Estas columnas poseen muchos valores faltantes, y no dan mucha información tanto categórica como cuantitativa para la ubicación del accidente, cosa que sí realizan otras columnas, así que no las usaré.  
**DIRECCIÓN NORMALIZADA, LUGAR DEL HECHO, XY(CABA) y PARTICIPANTES:** La información de estas columnas, ya se encuentra en otras columnas y estas son más que una relación entre estas columnas, como por ejemplo la columna participantes es la relación entre victima y acusado, por lo que conservaré las otras columnas para los objetivos del análisis.  
**FECHA_FALLECIMIENTO:** Esta columna no nos aportará al análisis.

Las columnas **XY(CABA)** y **Participantes**, tienen menos valores nulos que las columnas que ellas mismas relacionan, por lo que la usaremos primero para recuperar la información que podamos de dichas columnas relacionadas y luego las eliminaremos.

In [7]:
df_h = df_h.drop(columns=['FECHA', 'DD', 'HORA', 'LUGAR_DEL_HECHO', 'Altura', 'Cruce', 'Dirección Normalizada'])
df_v = df_v.drop(columns=['FECHA', 'DD', 'FECHA_FALLECIMIENTO'])

Veamos que columnas podemos recuperar de la posición X o Y.

In [8]:
df_h[(df_h['pos x'].isna()) | (df_h['pos y'].isna())].head(3) # Listamos los datos que tengan faltantes en X o en Y.

,ID,N_VICTIMAS,AAAA,MM,HH,TIPO_DE_CALLE,Calle,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1,2016,4,20,AUTOPISTA,"LUGONES, LEOPOLDO AV.",13,Point (. .),<NA>,<NA>,MOTO-SD,MOTO,<NA>
106,2016-0136,1,2016,10,0,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,4,Point (. .),<NA>,<NA>,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016,11,20,CALLE,NaN,0,Point (. .),<NA>,<NA>,PEATON-SD,PEATON,<NA>


Como podemos ver, ningún XY(CABA) posee información sobre las variables X o Y, por lo que dejaremos esta columna fuera como estaba planeado en un comienzo. Observemos si pasa lo mismo con la columna participantes.

In [9]:
df_h[(df_h['PARTICIPANTES'].isna()) | (df_h['ACUSADO'].isna())].head(3)

,ID,N_VICTIMAS,AAAA,MM,HH,TIPO_DE_CALLE,Calle,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
3,2016-0004,1,2016,1,0,AVENIDA,LARRAZABAL AV.,8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,<NA>
32,2016-0045,1,2016,4,10,GRAL PAZ,"PAZ, GRAL. AV.",13,Point (99620.34936816 110483.29286598),-58.46743471,-34.53476874,MOTO-SD,MOTO,<NA>
35,2016-0049,1,2016,4,0,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,<NA>,<NA>


Como podemos observar, esta columna tampoco posee información que las columnas Victima y Acusado no posean, por lo que también la eliminaremos.  
**Nota:** Limité los registros a 3 para una mejor visualización en el notebook, pero todos ellos cumplen con lo que afirmé.

In [10]:
df_h = df_h.drop(columns=['XY (CABA)', 'PARTICIPANTES'])

Miremos ahora el total de nulos en nuestras columnas de interés.

In [11]:
df_h.isna().sum()

ID                0
N_VICTIMAS        0
AAAA              0
MM                0
HH                1
TIPO_DE_CALLE     0
Calle             1
COMUNA            0
pos x            12
pos y            12
VICTIMA           9
ACUSADO          23
dtype: int64

Debido a la poca cantidad de registros nulos que nos quedaron en las demás columnas, siendo 23 el máximo para una de ellas, no se realizará una investigación más profunda para recuperar estos registros, ya que son una muy pequeña parte de los 696 datos del total.

In [12]:
df_h.dropna(inplace=True)

In [13]:
df_v.isna().sum()

ID_hecho     0
AAAA         0
MM           0
ROL         11
VICTIMA      9
SEXO         6
EDAD        53
dtype: int64

Se repite lo mismo para los datos de victimas, pero de una manera distinta, ya que los datos de EDAD y SEXO se realizarán para gráficos de distribución independientes, por lo que sus nulos no afectarán a los KPI que expondremos.

In [14]:
df_v.dropna(subset=['VICTIMA'], inplace=True)

# 3. Transformación columnas y datos
Para un mejor entendimiento de los datos, renombraré ciertas columnas o datos que puedan aparecer en ellas.

## 3.1 Columnas

Revisemos qué columnas necesitan un cambio de nombre.

In [15]:
df_h.columns

Index(['ID', 'N_VICTIMAS', 'AAAA', 'MM', 'HH', 'TIPO_DE_CALLE', 'Calle',
       'COMUNA', 'pos x', 'pos y', 'VICTIMA', 'ACUSADO'],
      dtype='object')

Renombramos las columnas que lo requieran.

In [16]:
df_h.rename(columns={'AAAA':'Anio', 'MM':'Mes', 'HH':'Hora', 'N_VICTIMAS':'NVictimas', 'TIPO_DE_CALLE':'TipoCalle', 'VICTIMA':'Victima', 'ACUSADO':'Acusado', 'COMUNA':'Comuna'}, inplace=True)

Ahora para el set de victimas.

In [17]:
df_v.columns

Index(['ID_hecho', 'AAAA', 'MM', 'ROL', 'VICTIMA', 'SEXO', 'EDAD'], dtype='object')

Las mismas columnas coinciden para necesitar un cambio de nombre.

In [18]:
df_v.rename(columns={'ID_hecho':'IdHecho', 'AAAA':'Anio', 'MM':'Mes', 'VICTIMA':'Victima', 'SEXO':'Sexo', 'EDAD':'Edad', 'ROL':'Rol'}, inplace=True)

## 3.2 Datos

Algunos datos que pueden aparecer en columnas, como es el caso de la columna VICTIMA o ACUSADO, que pueden tener datos como PASAJEROS, en el cual no se refieren a pasajeros como tal sino a un vehículo que transporta pasajeros como un Bus, los reemplazaremos por una palabra de más entendimiento, y a su vez, transformaremos las columnas necesarias a minúsculas para una mejor lectura y búsqueda de los datos.

In [19]:
df_h.info()

<class 'pandas.core.frame.DataFrame'>
Index: 659 entries, 0 to 695
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         659 non-null    object
 1   NVictimas  659 non-null    int64 
 2   Anio       659 non-null    int64 
 3   Mes        659 non-null    int64 
 4   Hora       659 non-null    object
 5   TipoCalle  659 non-null    object
 6   Calle      659 non-null    object
 7   Comuna     659 non-null    int64 
 8   pos x      659 non-null    object
 9   pos y      659 non-null    object
 10  Victima    659 non-null    object
 11  Acusado    659 non-null    object
dtypes: int64(4), object(8)
memory usage: 66.9+ KB


In [20]:
cats = ['TipoCalle', 'Calle', 'Victima', 'Acusado'] # Nuestras columnas categóricas de interés.
df_h[cats] = df_h[cats].map(str.lower) # Convertimos las columnas a minúsculas.

Reemplazaremos ahora unas palabras clave para un mejor entendimiento. Como por ejemplo las Victimas o Acusados de tipo pasajeros, son en realidad autobuses de transporte de pasajeros, los datos de otro, son vehículos de emergencia según la hoja llamada clas en el Excel, al igual que los movil. También hay 1 caso de peaton_moto, el cual es posiblemente un peatón transportando una moto en las manos, que será simplemente un peatón.

In [21]:
reemplazos = {'cargas': 'de_carga', 'pasajeros': 'autobus', 'otro': 'emergencias', 'movil': 'emergencias', 'peaton_moto': 'peaton'} # Definimos las palabras que vamos a reemplazar.
df_h.replace(reemplazos, inplace=True) # Reemplazamos.

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_26472\719601736.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_h.replace(reemplazos, inplace=True) # Reemplazamos.


Ahora realizamos lo mismo con los datos de las Víctimas.

In [22]:
df_v.info()

<class 'pandas.core.frame.DataFrame'>
Index: 708 entries, 0 to 716
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   IdHecho  708 non-null    object
 1   Anio     708 non-null    int64 
 2   Mes      708 non-null    int64 
 3   Rol      705 non-null    object
 4   Victima  708 non-null    object
 5   Sexo     704 non-null    object
 6   Edad     659 non-null    object
dtypes: int64(2), object(5)
memory usage: 44.2+ KB


In [23]:
df_v['Victima'] = df_v['Victima'].str.lower()
df_v['Rol'] = df_v['Rol'].str.lower()

In [24]:
df_v.replace(reemplazos, inplace=True)

In [25]:
df_v.replace({'MASCULINO':'M', 'FEMENINO':'F'}, inplace=True)

# 4. Limpieza profunda

## 4.1 Homicidios

**Advertencia:** Para evitar errores en la re-ejecución del Notebook, todas las columnas de mapeo serán comentadas, si las quieres ejecutar puedes seleccionar toda la celda de código y usar CTRL + / para des-comentar. Luego, lee los comentarios y los print guía para reemplazar los valores. Para realizar el mapeo automáticamente al ejecutar el notebook, se guardó un JSON con el último mapeo realizado que no dió errores, y dicho mapeo se leerá y aplicará automáticamente al set de datos.

Eliminemos posibles errores de escritura de la columna Calle, también reemplacemos valores de ñ y tildes desconocidos.

In [26]:
# mapeo = {} # Guardamos los valores que cambiaremos.
# for elm in (df_h['Calle'].unique()): # Iteramos en cada valor único de Calle.
#     old = elm # El nombre del elemento lo almacenaremos, ya que lo cambiaremos.
#     while '?' in elm: # Mientras que haya un carácter desconocido en el elemento, lo trataremos manualmente para saber de qué carácter se trata.
#         rep = input(f'Replace for {elm}:') # Ingresamos el reemplazo.
#         elm = elm.replace('?', rep, 1) # Reemplazamos la primera coincidencia.
#     if elm != old: # Si el elemento cambió, lo agregamos a el mapeo que usaremos.
#         mapeo[old] = elm
# mapeo # Mostramos el mapeo para verificar.

{'magari?os cervantes, a.': 'magariños cervantes, a.',
 'nu?ez': 'nuñez',
 'casta?ares av.': 'castañares av.',
 'albari?o': 'albariño',
 'autopista presidente h?ctor j. c?mpora': 'autopista presidente héctor j. cámpora',
 'mu?ecas': 'muñecas',
 'rodriguez pe?a': 'rodriguez peña'}

In [27]:
# df_h.replace(mapeo, inplace=True) # Reemplazamos las calles que no se leyeron bien por los valores mapeados previamente.

Ahora, tratemos los errores de escritura que pudieron tener las calles, primero los detectaremos con una función y luego los reemplazaremos si es necesario.

In [28]:
# # Extraemos las avenidas y las convertimos en un DataFrame para solo iterar en las primeras filas siguientes a ellas.
# # creamos una serie temporal, ordenado, y retornamos sus valores únicos.
# streets = (sorted(df_h['Calle'].unique())) 
# # En base a la serie, creamos un DataFrame, para iterar con más comodidad en sus indices.
# streets = pd.DataFrame({'Calle': streets})
# # Con esta función encontraremos el puntaje que le da el SequenceMatcher basando en la similitud de las cadenas.
# def similar(a, b):
#     return SequenceMatcher(None, a, b).ratio() # Retornamos la similitud entre las cadenas de texto A y B.

# # La función agrupar texto, recibe un DataFrame, una columna objetivo, y un límite, el cual definirá qué tanto se deben parecer las cadenas,
# # para resultados diferentes modificar el valor de threshold, especificándolo en los parámetros de la función.
# def agrupar_texto(df, col, threshold=0.80):
#     # Guardamos en un diccionario los valores que se lograron agrupar.
#     agrupados = {}
#     # Guardamos los ya categorizados para evitar redundancias.
#     done = []
#     # Iteramos en el indice y las filas de el DataFrame ingresado en la función.
#     for i, row in df.iterrows():
#         # Limitar la comparación solo a los siguientes 10 registros.
#         for j in range(i + 1, min(i + 11, len(df))):
#             # Localizamos el elemento en la posición J en la columna ingresada.
#             elm = df.loc[j, col]
#             # Calculamos la similitud.
#             similitud = similar(row[col], elm)
#             # Si es mayor al límite, agregamos el elemento a el grupo.
#             if similitud > threshold:
#                 if not(elm in done):
#                     agrupados.setdefault(row[col], []).append(elm)
#                 done.append(elm)
#     return agrupados

# # Agrupamos los desarrolladores con nuestra función.
# similares = agrupar_texto(streets, 'Calle')
# similares

Al ser pocos registros, los reemplazaremos manualmente pasando sobre cada uno y decidiendo en cada uno, ya que hay algunos que no deben ser reemplazados, pero si aumentamos el límite de detección de la función, no detectará otros que sí deberían serlo.

In [29]:
# # Un diccionario temporal para evitar errores en la iteración.
# temp = similares.copy()
# # Iteramos en las llaves y valores.
# for k, v in similares.items():
#     # Confirmamos si se quieren reemplazar o cambiar el valor final que tendrá.
#     print(f"Se reemplazaran: {v} por '{k}'.")
#     print("1. Confirmar\n2. Cambiar valor final\n0. No Cambiar")
#     op = int(input())
#     # Para no cambiar el valor se ingresa 0, esto omitirá ese cambio.
#     if op == 0:
#         del temp[k]
#     # Para modificar el valor final, ingresa 2 y luego ingresa la posición en la que está la cadena que se mantendrá.
#     elif op == 2:
#         pos = int(input(f"¿En qué posición está el elemento que quiere reemplazar?, cuente desde 1.\nLista: {v}"))
#         # Creamos la nueva llave.
#         new = temp[k][pos-1]
#         # Eliminamos el valor de la lista.
#         v.remove(new)
#         # Agregamos el valor antiguo a la lista.
#         v.append(k)
#         # Eliminamos la antigua llave.
#         del temp[k]
#         # Creamos con la nueva llave y la lista con el valor anterior incluido, un nuevo elemento en el mapeo.
#         temp[new] = v
#     clear_output()
# temp

Creamos la función que mapeara el DataFrame.

In [29]:
def mapear(df, temp):
    # Iteramos sobre los elementos y llaves de nuestro mapeo previamente creado.
    for k, elms in temp.items():
        # Iteramos sobre cada elemento a reemplazar.
        for elm in elms:
            # Reemplazamos el elemento por la llave en el DataFrame.
            df.replace(elm, k, inplace=True)

Mapeamos o lanzamos una advertencia si no existe ningún mapeo previo o actual.

In [30]:
path = '../Data/Processed/last_map.json' # Ruta del archivo.
try:
    # Intentamos mapear con un mapeo actual, si se ejecutaron las celdas anteriores.
    mapear(df_h, temp)
    # Guardamos el mapeo creado para su posterior uso.
    with open(path, 'w', encoding='utf-8') as file:
        json.dump(temp, file)
# Si no se ejecutaron, intentaremos buscar un mapeo previo.
except NameError:
    print("No se ha realizado un mapeo, se intentará cargar uno.")
    if os.path.exists(path):
        # Si existe un mapeo previo, lo usaremos.
        with open(path, 'r', encoding='utf-8') as file:
            temp = json.load(file)
            mapear(df_h, temp)
            print("El mapeo se cargó y se realizó exitosamente.")
    # Si todo falló, describiremos lo que ocurrió.
    else:
        print("No se encontró un mapeo previo, realiza uno para hallar los errores de escritura en las variables.")

No se ha realizado un mapeo, se intentará cargar uno.
El mapeo se cargó y se realizó exitosamente.


## 4.2 Victimas

Aún debemos limpiar los valores de Victimas que no tengan registros en Homicidios, ya que no tendremos información de ellos.  
También asignaremos el rol de conductor a las victimas que solo tengan 1 cantidad total de victimas y que su rol sea nulo, ya que es muy probable que haya sido el conductor el implicado en el homicidio.

In [32]:
df_v.loc[(df_v['Rol'].isna()) & ~(df_v.duplicated()), 'Rol'] = 'conductor' # Reemplazamos los roles que sean nulos, y que a su vez no tengan duplicados, por el rol de conductor.

Ahora eliminamos los registros de los que no se tenga información.

In [33]:
df_v = df_v[(df_v['IdHecho'].isin(df_h['ID']))] # Filtramos el DataFrame incluyendo únicamente los registros que tengan un ID presente en homicidios.

Verificamos los nulos

In [35]:
df_v.isna().sum()

IdHecho     0
Anio        0
Mes         0
Rol         0
Victima     0
Sexo        1
Edad       43
dtype: int64

# 5. Exportamos los datos

Previsualizamos los datos para ver el resultado de la limpieza.

In [36]:
df_h.head()

,ID,NVictimas,Anio,Mes,Hora,TipoCalle,Calle,Comuna,pos x,pos y,Victima,Acusado
0,2016-0001,1,2016,1,4,avenida,piedra buena av.,8,-58.47533969,-34.68757022,moto,auto
1,2016-0002,1,2016,1,1,gral paz,"paz, gral. av.",9,-58.50877521,-34.66977709,auto,autobus
2,2016-0003,1,2016,1,7,avenida,entre rios av.,1,-58.39040293,-34.63189362,moto,auto
4,2016-0005,1,2016,1,5,avenida,san juan av.,1,-58.38718297,-34.62246630,moto,autobus
5,2016-0008,1,2016,1,18,avenida,27 de febrero av.,8,-58.44451316,-34.68475866,moto,objeto fijo


In [37]:
df_v.head()

,IdHecho,Anio,Mes,Rol,Victima,Sexo,Edad
0,2016-0001,2016,1,conductor,moto,M,19
1,2016-0002,2016,1,conductor,auto,M,70
2,2016-0003,2016,1,conductor,moto,M,30
4,2016-0005,2016,1,conductor,moto,M,29
5,2016-0008,2016,1,conductor,moto,M,30


Eliminamos las columnas redundantes de las victimas.

In [38]:
df_v.drop(columns=['Anio', 'Mes', 'Victima'], inplace=True)

Exportamos cada DataFrame para su uso en el EDA.

In [39]:
df_h.to_csv('../Data/Processed/homicidios_clean.csv', index=False, encoding='utf-8')
df_v.to_csv('../Data/Processed/victimas_clean.csv', index=False, encoding='utf-8')